In [5]:
import numpy as np
from sklearn.decomposition import PCA
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split


In [7]:
#Old upload for all data

y_path = Path.cwd().parent.joinpath("Data","nao_index_train.npy")
tas_path = Path.cwd().parent.joinpath("Data","tas_train.npy") #temperature average of autmn per year
tsl_path= Path.cwd().parent.joinpath("Data","psl_train.npy") #sea level pressure average of autmn per year


y_train=np.load(y_path)
tas_train=np.load(tas_path)
tsl_train=np.load(tsl_path)


In [8]:
tas_path

PosixPath('/p/home/jusers/goergen2/jusuf/weatherpeople/Data/tas_train.npy')

In [9]:
tas_train.shape

(900, 2322)

In [10]:
#define parameters here
n_comp=50 #how many principal components do you want

In [11]:
pca= PCA(n_components=n_comp)

In [12]:
temp_trans= pca.fit_transform(tas_train)

In [13]:
print(pca.explained_variance_ratio_)

[0.139324   0.08886949 0.08387699 0.06725111 0.05784979 0.04628902
 0.04110706 0.03692171 0.0271803  0.02323685 0.02261667 0.02123957
 0.01944782 0.01610227 0.01520486 0.01439683 0.01356993 0.0122921
 0.01148158 0.01075819 0.00956839 0.00779558 0.0072168  0.00705343
 0.00634401 0.00613418 0.00562966 0.00522838 0.00477947 0.00455712
 0.00425549 0.00418571 0.00402752 0.0039618  0.00360039 0.00336402
 0.00314784 0.00312429 0.00296509 0.00289858 0.00271183 0.00261067
 0.00256189 0.00246326 0.00237054 0.00227419 0.0021777  0.002122
 0.00208886 0.00196739]


In [14]:
pca_tsl=PCA(n_components=n_comp)
tsl_trans = pca_tsl.fit_transform(tsl_train)

In [15]:
print(pca_tsl.explained_variance_ratio_)

[0.30490791 0.18400934 0.07381419 0.06671203 0.06166861 0.05263237
 0.03733528 0.0288457  0.02491261 0.01680746 0.01616275 0.01394154
 0.01269437 0.01039641 0.00871227 0.00683595 0.0065405  0.00577146
 0.00498761 0.00454754 0.00414056 0.0035176  0.00308141 0.00295469
 0.00262907 0.00232871 0.00223747 0.00216999 0.00185604 0.00157332
 0.00155057 0.00140081 0.0013316  0.00123748 0.0011494  0.00108885
 0.00104446 0.0009467  0.00085657 0.00080332 0.00074315 0.00071773
 0.00067387 0.00063728 0.00059264 0.00057116 0.00054763 0.0005225
 0.00047921 0.00046028]


In [16]:
print(pca_tsl.explained_variance_ratio_.sum())

0.9860799899628351


In [3]:
#Random forest first try
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import sklearn.metrics
from sklearn.model_selection import GridSearchCV




In [14]:
from imblearn.over_sampling import RandomOverSampler 

In [57]:
#Random Forest with CV

PCA(n_components=50)


In [18]:
#old code
pca_train=np.concatenate([temp_trans, tsl_trans], axis=1)
binary_labels = np.sign(y_train)

In [19]:
rf_def = RandomForestClassifier(random_state=1337,criterion="entropy") 

In [20]:
rf_def.fit(pca_train,binary_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1337,
                       verbose=0, warm_start=False)

In [ ]:
#use f1 score, i.e. f1=2 * (Precision * Recall) / (Precision + Recall)

scores=cross_val_score(rf_def, pca_train, binary_labels, cv=10,n_jobs=-1,scoring=make_scorer(f1_score))

In [ ]:
scores.mean()

In [34]:
#Random Forest with test set without PCA:

y_path = Path.cwd().parent.joinpath("Data","Split","train_labels.npy")
x_path = Path.cwd().parent.joinpath("Data","Split","train_features.npy")
x_test_path = Path.cwd().parent.joinpath("Data","Split","test_features.npy")
y_test_path = Path.cwd().parent.joinpath("Data","Split","test_labels.npy")

y_train = np.load(y_path)
x_train = np.load(x_path)
y_test = np.load(y_test_path)
x_test = np.load(x_test_path)

In [ ]:
#get data into format you want
y_train = np.sign(y_train)
y_test = np.sign(y_train)



In [35]:
#categorial
med_ids = np.where(np.logical_and(-1.5<y_train, y_train<1.5))[0]
y_train[y_train>=1.5] = 1. 
y_train[y_train<=-1.5] = -1.
y_train[med_ids] = 0.

#same for y_test

med_ids_test = np.where(np.logical_and(-1.5<y_test, y_test<1.5))[0]
y_test[y_test>=1.5] = 1. 
y_test[y_test<=-1.5] = -1.
y_test[med_ids_test] = 0.

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [7]:
rf_split = RandomForestClassifier(random_state=1337,criterion="entropy") 
rf_split.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', random_state=1337)

In [8]:
np.shape(x_train)

(800, 4644)

In [13]:
scores=cross_val_score(rf_split, x_train, y_train, cv=10,n_jobs=-1,scoring=make_scorer(f1_score))

In [14]:
scores.mean()

0.6203836555524201

In [10]:
rf_split.predict(x_test)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
        1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
        1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,
        1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,
        1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.])

In [12]:
scores_oos = f1_score(y_test,rf_split.predict(x_test))

In [13]:
scores_oos

0.5967741935483871

In [36]:
#Use oversampling

ros = RandomOverSampler(random_state=1337)
X_resampled, y_resampled = ros.fit_resample(x_train, y_train)

In [37]:
np.shape(X_resampled)

(2067, 4644)

In [38]:
np.shape(x_train)

(800, 4644)

In [39]:
rf_resample = RandomForestClassifier(random_state=1337,criterion="entropy") 
rf_resample.fit(X_resampled,y_resampled)

RandomForestClassifier(criterion='entropy', random_state=1337)

In [48]:
scores_oos = f1_score(y_test,rf_split.predict(x_test),average="micro")

In [49]:
scores_oos

0.10000000000000002

In [ ]:
#some more testts